<a href="https://colab.research.google.com/github/AABarodkin/python_mftids22/blob/main/%22Environment_ver1_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [332]:
!pip install -q -U kaggle_environments

In [333]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

Агент#1 который всегда камень

In [334]:
%%writefile rock_agent.py


#0 - rock
#1 - paper
#2 - scissors
def your_agent(observation, configuration):
    return 0

Overwriting rock_agent.py



Агент#2 который всегда ножницы

In [335]:
%%writefile scissors_agent.py


#0 - rock
#1 - paper
#2 - scissors
def your_agent(observation, configuration):
    return 2

Overwriting scissors_agent.py


Агент#3 котрый всегда бумага

In [336]:
%%writefile paper_agent.py


#0 - rock
#1 - paper
#2 - scissors
def your_agent(observation, configuration):
    return 1

Overwriting paper_agent.py


Агент#4, который производит то же самое действие, что и оппонент на прошлом ходу

In [337]:
%%writefile copy_opponent.py

import random
def copy_opponent(observation, configuration):

    if observation.step > 0:
        return observation.lastOpponentAction
    #initial step
    else:
        return random.randrange(0, configuration.signs)

Overwriting copy_opponent.py


Агент#5 который делает победное действие прошлого хода оппонента

In [338]:
%%writefile wincopy_opponent.py

import random
def wincopy_opponent(observation, configuration):

    if observation.step > 0:
      if observation.lastOpponentAction==0:
        return 1
      if observation.lastOpponentAction==1:
        return 2
      if observation.lastOpponentAction==2:
        return 0
    else:
        return random.randrange(0, configuration.signs)

Overwriting wincopy_opponent.py


Агент #6 который делает проигрошное действие прошлого хода оппонента

In [339]:
%%writefile losscopy_opponent.py
#0 - rock
#1 - paper
#2 - scissors
import random
def losscopy_opponent(observation, configuration):

    if observation.step > 0:
      if observation.lastOpponentAction==0:
        return 2
      if observation.lastOpponentAction==1:
        return 0
      if observation.lastOpponentAction==2:
        return 1
    else:
        return random.randrange(0, configuration.signs)

Overwriting losscopy_opponent.py


Агент #7 который статистично оценивает какое у оппонента наиболее частое действие за весь период игры и ставит всегда выйгрышное действие против него

In [340]:
%%writefile statist_opponent.py
#0 - rock
#1 - paper
#2 - scissors
import random
stat={0:0,1:0,2:0}
def statist(observation, configuration):
  global stat
  if observation.step > 0:
    stat[observation.lastOpponentAction]=stat.get(observation.lastOpponentAction,0)+1
    k=[stat[0],stat[1],stat[2]]
    if max(k)==0:
      return 1
    if max(k)==1:
      return 2
    if max(k)==2:
      return 0
  else:
    return random.randrange(0, configuration.signs)

Overwriting statist_opponent.py


Агент #8 который статистично оценивает какое у оппонента наиболее редкое действие за весь период игры и ставит всегда выйгрышное действие против него

In [341]:
%%writefile minstatist_opponent.py
#0 - rock
#1 - paper
#2 - scissors
import random
stat={0:0,1:0,2:0}
def minstatist(observation, configuration):
  global stat
  if observation.step > 0:
    stat[observation.lastOpponentAction]=stat.get(observation.lastOpponentAction,0)+1
    k=[stat[0],stat[1],stat[2]]
    if min(k)==0:
      return 1
    if min(k)==1:
      return 2
    if min(k)==2:
      return 0
  else:
    return random.randrange(0, configuration.signs)

Overwriting minstatist_opponent.py


Агент #9 рандомный агент

In [342]:
%%writefile random_agent.py

import random
#0 - rock
#1 - paper
#2 - scissors
def your_agent(observation, configuration):
    return random.randrange(0, configuration.signs)

Overwriting random_agent.py


Агент #10 рандомный агент камень бумага

In [343]:
%%writefile randomRP_agent.py

import random
#0 - rock
#1 - paper
#2 - scissors
def your_agent(observation, configuration):
    return random.randint(0, 1)

Overwriting randomRP_agent.py


Агент #11 рандомный агент камень бумага

In [344]:
%%writefile randomPS_agent.py

import random
#0 - rock
#1 - paper
#2 - scissors
def your_agent(observation, configuration):
    return random.randint(1, 2)

Overwriting randomPS_agent.py


Агент #12 рандомный агент камень ножницы

In [345]:
%%writefile randomRS_agent.py

import random
#0 - rock
#1 - paper
#2 - scissors
def your_agent(observation, configuration):
    return 2*random.randint(0, 1)

Overwriting randomRS_agent.py


Создадим список всех агентов

In [346]:
agents=["rock_agent.py", "scissors_agent.py", "paper_agent.py", "copy_opponent.py", "wincopy_opponent.py", "losscopy_opponent.py", 'statist_opponent.py', 'minstatist_opponent.py', 'random_agent.py', 'randomRP_agent.py', 'randomPS_agent.py', 'randomRS_agent.py']

Создадим пустой список для подсчета очков

In [347]:
scores=[0 for x in range(12)]


Создадим пустую таблицу для занесения результатов персональных дуэлей

In [348]:
table=[scores for x in range(12)]


In [349]:
df = pd.DataFrame(table, agents, agents)



In [350]:
for i in range(len(agents)):
  for j in range(i+1, len(agents)):
    result=evaluate("rps", [agents[i], agents[j]], configuration={"episodeSteps": 100})
    df[agents[i]][agents[j]]=result[0][0]
    df[agents[j]][agents[i]]=result[0][1]  

Таблица соревнований (в столбцах указан результат для участника (наименования столбца)). Для удобства добавлена строка Total с итоговым результатом для каждого участника.

In [351]:
df.loc['Total']=[0 for x in range(12)]
for i in agents:
  df.loc['Total', i]=df[i].sum()
df


,rock_agent.py,scissors_agent.py,paper_agent.py,copy_opponent.py,wincopy_opponent.py,losscopy_opponent.py,statist_opponent.py,minstatist_opponent.py,random_agent.py,randomRP_agent.py,randomPS_agent.py,randomRS_agent.py
rock_agent.py,0,-99,99,0,99,-97,0,99,0,48,0,-52
scissors_agent.py,99,0,-99,0,98,-98,97,-97,0,0,-53,55
paper_agent.py,-99,99,0,0,99,-98,-95,0,0,-58,46,0
copy_opponent.py,0,0,0,0,-99,99,0,0,0,0,0,0
wincopy_opponent.py,-99,-98,-99,99,0,-99,-94,-94,0,-21,0,-23
losscopy_opponent.py,97,98,98,-99,99,0,95,97,0,33,0,26
statist_opponent.py,0,-97,95,0,94,-95,0,95,0,51,0,-50
minstatist_opponent.py,-99,97,0,0,94,-97,-95,0,0,-38,44,0
random_agent.py,0,0,0,0,0,0,0,0,0,0,0,0
randomRP_agent.py,-48,0,58,0,21,-33,-51,38,0,0,20,0


Выводим результаты:

In [352]:
winner=''

In [353]:
for i in agents:
  print(i, ':', df.loc['Total', i])
  if df.loc['Total', i]==df.loc['Total'].max():
    winner=i

rock_agent.py : -97
scissors_agent.py : -2
paper_agent.py : 106
copy_opponent.py : 0
wincopy_opponent.py : 528
losscopy_opponent.py : -544
statist_opponent.py : -93
minstatist_opponent.py : 94
random_agent.py : 0
randomRP_agent.py : -5
randomPS_agent.py : 36
randomRS_agent.py : -23


Победитель!

In [354]:
print('Winner:', winner, 'with:' , df.loc['Total', winner])

Winner: wincopy_opponent.py with: 528


wincopy_opponent.py всегда выигрывает у первых трех оппонентов и у 6,7,8, из за быстрого подстроения к их стратегии. С рандомными (последними 4 - 50/50 или в небольшой плюс) здесь мало что зависит от его стратегии) Чем задает себе задел на победу!